In [2]:
import json
import pandas as pd
import numpy as np

In [13]:
from keras.models import load_model

model = load_model('predict_match_model.keras')

In [11]:
from joblib import load

scaler = load('scaler.pkl')

In [3]:
# Load the JSON data
with open('updated_teams_with_history_and_elo.json', 'r') as file:
    teams_data = json.load(file)

In [4]:
from datetime import datetime

In [5]:
teams_dict = {
    team['team_id']: {k: v for k, v in team.items() if k != 'team_id'}
    for team in teams_data
}

In [9]:
def top_5_elo_sum(players_elo):
    # Filter out None values and take top 5 player elos
    
    if len(players_elo) == 0:
        return [0]*4
    sum_elos = []
    for year in range(4):
        valid_elos = [player["elo"][year] for player in players_elo if player["elo"] != None]
        sum_elos.append(sum(sorted(valid_elos, reverse=True)[:min(5,len(valid_elos))]))
    return sum_elos

In [19]:
features_list = ['inhibKills', 'towerKills', 'baronKills', 'dragonKills', 'Gold_per_min', 'KA_per_min', 'deaths_per_min', 'game_length', 'Gold_15', 'towerKills_15', 'dragonKills_15', 'KA_15', 'deaths_15', 'GD_15', 'GPMD', 'towerKillsD_15', 'dragonKillsD_15', 'KAD_15', 'players_elo', 'team_elo']

In [20]:
# Helper function to create a feature vector for a given team ID
def create_feature_vector(team_id1, team_id2):
    for data in teams_data:
        
        if team_id1 == data["team_id"]:
            
            # for game in data["match_history"]:
                
    
            feature_vector = {}
        
            # Extract all matches between the two teams
            matches = [match for match in data["match_history"] if match["Opponent_team"] == team_id2]
            
            if not matches:
                  # Get the column names from the DataFrame
                zero_match = {key: 0 for key in features_list}
                opponent_players_elo = top_5_elo_sum(teams_dict[team_id2]['players'])[3] if team_id2 in teams_dict.keys() else 0
                player_elo = top_5_elo_sum(data["players"])[3]
                zero_match["players_elo"] = player_elo/max(player_elo+opponent_players_elo,1)
                
                opponent_team_elo = teams_dict[team_id2]['elo'][3] if team_id2 in teams_dict.keys() else 0
                team_elo = data["elo"][3]
                zero_match["team_elo"] = team_elo/(team_elo+opponent_team_elo)
                df_to_predict = pd.DataFrame([zero_match])
                return df_to_predict
            # Initialize a dictionary to store the sum of each statistic
            stats_sum = {k: 0 for k in matches[0].keys() if k not in ["team_elo","players_elo","Result", "eventTime", "year", "Team_id", "Opponent_team", "LeagueId"]}
            
            # # Extract latest match history details excluding "Result"
            # latest_match = [match for match in data["match_history"] if match["Opponent_team"] == team_id2]
            # latest_match_features = {k: v for match in latest_match for k, v in match.items() if k not in ["Result","eventTime","year","Team_id","Opponent_team","LeagueId"]}

            
            
            # Iterate over matches and accumulate the statistics
            for match in matches:
                for key in stats_sum.keys():
                    stats_sum[key] += match[key]

            # Compute the average for each statistic
            average_stats = {k: v / len(matches) for k, v in stats_sum.items()}
            
            
            
            # Merge with the feature vector
            feature_vector.update(average_stats)
            # if len(feature_vector.keys()) == 0:
            #     feature_vector = {k: 0 for k in match_key if k not in ["Result","eventTime","year","Team_id","Opponent_team","LeagueId"]} 
            opponent_players_elo = top_5_elo_sum(teams_dict[team_id2]['players'])[3] if team_id2 in teams_dict.keys() else 0
            player_elo = top_5_elo_sum(data["players"])[3]
            feature_vector["players_elo"] = player_elo/max(player_elo+opponent_players_elo,1)
            
            opponent_team_elo = teams_dict[team_id2]['elo'][3] if team_id2 in teams_dict.keys() else 0
            team_elo = data["elo"][3]
            feature_vector["team_elo"] = team_elo/(team_elo+opponent_team_elo)
            df_to_predict = pd.DataFrame([feature_vector])
    return scaler.transform(df_to_predict)



In [57]:
def predict_match(team1_id, team2_id):    
    # Create feature vectors for the two provided teams
    # team_1_vector = create_feature_vector("99566404852189289","107251295460371069")
    team_1_vector = create_feature_vector(team1_id,team2_id)
    team_2_vector = create_feature_vector(team2_id,team1_id)
    
    # Convert to DataFrame for prediction
    df_predict1 = pd.DataFrame(team_1_vector)
    df_predict2 = pd.DataFrame(team_2_vector)
    # Predict using the trained model
    # predictions = clf.predict(df_predict)
    predictions = model.predict(df_predict1),model.predict(df_predict2)
    # predictions
    
    predicted_classes = np.argmax(predictions, axis=0)
    class_index = int(predicted_classes[0][0])  # This should give you a scalar value
    team_id = [team1_id, team2_id][class_index]
    
    prob = predictions[class_index] / sum(predictions)

    return team_id, prob[0][0]

In [60]:
predict_match("98767991954244555","99566404585387054") # GAM vs DRX. DRX wins

1/1 [==============================] - 0s 20ms/step


('99566404585387054', 0.9488448)

In [59]:
predict_match("100725845018863243","107563714667537640") # DK vs BDS. DK wins

1/1 [==============================] - 0s 22ms/step


('100725845018863243', 0.5053629)

In [62]:
predict_match("100725845018863243","100205573495116443") # DK vs GEN. GEN wins

1/1 [==============================] - 0s 20ms/step


('100205573495116443', 0.86138386)